## EZKL Jupyter Notebook Demo

Here we demonstrate the use of the EZKL package in a Jupyter notebook whereby all components of the circuit are public or pre-committed to. This is the simplest case of using EZKL (proof of computation).

In [1]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch


# Defines the model
# we got convs, we got relu, we got linear layers
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)

        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)

        return logits


#circuit = MyModel()

# Train the model as you like here (skipped for brevity)

def slerp(val, low, high):
    """
    Spherical interpolation between low and high tensors.
    """
    omega = torch.acos(torch.clamp(torch.sum(low/torch.norm(low)*high/torch.norm(high)), -1, 1))
    so = torch.sin(omega)
    return torch.sin((1.0-val)*omega) / so * low + torch.sin(val*omega) / so * high

class MetaModel(nn.Module):
    def __init__(self, model1, model2, merge_type='average'):
        """
        Initialize MetaModel with two models and merge type.
        :param model1: First model
        :param model2: Second model
        :param merge_type: Type of merging ('average' or 'slerp')
        """
        super(MetaModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model = MyModel()
        self.merge_type = merge_type
        self.m1_weight = 0.5
        self.m2_weight = 0.5

        self._merge_parameters()

    def _merge_parameters(self):
        for param1, param2, param in zip(model1.parameters(), model2.parameters(), self.model.parameters()):
            if self.merge_type == 'slerp':
                param.data.copy_(slerp(0.5, param1.data, param2.data))
            else:  # Default to linear merge
                param.data.copy_(self.m1_weight * param1.data + self.m2_weight * param2.data)

    def forward(self, x):
        return self.model(x)

# Instantiate two models
model1 = MyModel()
model2 = MyModel()

# Load pre-trained weights if available
# model1.load_state_dict(torch.load('path_to_model1_weights.pth'))
# model2.load_state_dict(torch.load('path_to_model2_weights.pth'))

# Instantiate the MetaModel with 'average' or 'slerp' merging
circuit = MetaModel(model1, model2, merge_type='slerp')



In [2]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [3]:

shape = [1, 28, 28]
# After training, export to onnx (network.onnx) and create a data file (input.json)
x = 0.1*torch.rand(1,*shape, requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


In [4]:

py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True


In [5]:
cal_path = os.path.join("calibration.json")

data_array = (torch.rand(20, *shape, requires_grad=True).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

ERROR:ezkl.execute:circuit creation from run args failed: TensorError(SigBitTruncationError)
ERROR:ezkl.execute:circuit creation from run args failed: TensorError(SigBitTruncationError)

 <------------- Numerical Fidelity Report (input_scale: 11, param_scale: 11, scale_input_multiplier: 1) ------------->

+-----------------+---------------+---------------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error      | median_error  | max_error     | min_error     | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-----------------+---------------+---------------+---------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.000050461353 | 0.00012412667 | 0.00061169267 | -0.0006187558 | 0.00018836439  |

True

In [6]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [7]:
# srs path
res = ezkl.get_srs( settings_path)

In [8]:
# now generate the witness file

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [10]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,

    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [11]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,

        "single",
    )

print(res)
assert os.path.isfile(proof_path)

{'instances': [['...']], 'proof': '0x0b2018ac4c64c06397d6f798c4ce1c18a575a79433a5d568c0579a2b92357ecb0d3ff38d52e5ad4944e1ea18660858a57a372b148d64069a65e288ff8b9f2ac127054ed959f6b84030cf0290704ebae4d37658d9f75c8c2fed0afd903e0430842e91cce46deed6cbe5ec41cf50aec654a43711f230f40296f601199da6fed9672dda43f8c09ea251f218884325a5281629bfbe9d1da3175019cd03eac7cc9ac00ad80e74b14474766e176b2b4b9c53a7336240c2c6e650a1cf90ded52be4f917063d428c9e0df9a327e629866dc011c64256f7d04c7b7c835e7d5b4733a55c5c1a97cf9a964de6927527a698a5c7ecccfbc34d7308cc927e0d8091551971a20d1a97d5ccf597c7ae3e4d86d94314002af92259f74a9ba96319c008a223f87cdc11018f09aaf414bbd2918133f5cb87187f58aa0161313f998aabbcc0b6855a1d071cf843f86ef8580797d1eceab33070e65b839d8cb6cce142d457e8b8ea1de6101d26ed3ee1cc816018be99ae5cbd1f7053977bbe8db9ece2fd556d72ebe03d2ad11437290c055351c741f2d18c821fdfff5f14de82ce30f9cfc90fd258804e221abcd9c64189a9a430fcb6d37a7a0f63773302bbe4c3424edf2b997cb4742028fa38dd17cfcef742f627c4f0ede32506895bc1f9790767474a5624435c92241d7

In [12]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,

    )

assert res == True
print("verified")

verified
